In [50]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.markers as mark
from matplotlib.lines import Line2D
import pandas as pd
import math

%matplotlib inline

cities = pd.read_csv('cities.csv')
cities = cities[cities['Population']>1e6]

wmap = mpimg.imread('Mercator_projection_Square.png')
height,width,c = wmap.shape

def conv_coord(dms):
    deg,min,sec = dms
    return deg + min/60 + sec/3600

r = width/2/math.pi

def merc_x(lon):
    return width/2 + r * math.radians(conv_coord(lon))

def merc_y(lat):
    phi = math.radians(conv_coord(lat))
    return height/2 - r * math.atanh(math.sin(phi))

def city_marker(v):    
    marker_obj = mark.MarkerStyle(v and '*' or 'o')
    return  marker_obj.get_path().transformed(marker_obj.get_transform())


cities['y'] = cities[['Longitude deg','Longitude min','Longitude sec']].apply(merc_y, axis=1)
cities['x'] = cities[['Latitude deg','Latitude min','Latitude sec']].apply(merc_x, axis=1)
cities['Marks'] = cities['Capital'].map(city_marker)

In [51]:
fig,ax = plt.subplots(figsize=(15,15),dpi=96)
ax.imshow(wmap)
ax.axis('off')

legend_el = []

for country, c_cities in cities.groupby('Country'):
    sc = ax.scatter(c_cities['x'],c_cities['y'])
    sc.set_paths(c_cities['Marks'].to_numpy())
    color = sc.get_facecolors()[0]
    legend_el.append(Line2D([0], [0], ls='None', marker='o', label=country, c=color ,ms=15))

plt.legend(handles=legend_el)
# plt.savefig('wmap.png')
plt.show()
